In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # type: ignore

# Thư viện cho Apriori
from mlxtend.frequent_patterns import apriori, association_rules # type: ignore

# Thư viện cho Clustering và Đánh giá
from sklearn.cluster import KMeans # type: ignore
from sklearn.metrics import silhouette_score # type: ignore
from sklearn.decomposition import PCA # type: ignore
from sklearn.preprocessing import StandardScaler # type: ignore

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')

print("Đã import xong thư viện.")

In [ ]:
# Đọc dữ liệu từ file CSV
try:
    df = pd.read_csv('HeartDiseaseTrain-Test.csv')
    print("Đọc dữ liệu thành công!")
    print(f"Kích thước bộ dữ liệu: {df.shape}")
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file csv. Hãy upload file 'HeartDiseaseTrain-Test.csv'.")

# Hiển thị 5 dòng đầu
df.head()

In [ ]:
# Tạo bản sao để xử lý
df_bin = df.copy()

# 1. Age (Tuổi): Chia làm 3 nhóm
# <45: Young, 45-60: Middle, >60: Old
df_bin['age_group'] = pd.cut(df['age'], bins=[0, 44, 60, 100], labels=['Age_Young', 'Age_Middle', 'Age_Old'])

# 2. Resting Blood Pressure (Huyết áp):
# <120: Normal, 120-139: Pre-High, >=140: High
df_bin['bp_group'] = pd.cut(df['resting_blood_pressure'], bins=[0, 120, 140, 300], labels=['BP_Normal', 'BP_Elevated', 'BP_High'])

# 3. Cholestoral (Mỡ máu):
# <200: Healthy, >=200: High
df_bin['chol_group'] = pd.cut(df['cholestoral'], bins=[0, 200, 600], labels=['Chol_Healthy', 'Chol_High'])

# 4. Max Heart Rate (Nhịp tim tối đa):
# Chia đều theo phân vị (Quantile) thành 3 nhóm: Low, Medium, High
df_bin['hr_group'] = pd.qcut(df['Max_heart_rate'], q=3, labels=['HR_Low', 'HR_Medium', 'HR_High'])

# 5. Oldpeak (Độ chênh ST):
# 0: No_Depression, >0: Depression
df_bin['oldpeak_group'] = df['oldpeak'].apply(lambda x: 'Oldpeak_No' if x == 0 else 'Oldpeak_Yes')

# 6. Target (Kết quả bệnh):
# Đổi số 0,1 thành nhãn để dễ đọc luật Apriori
df_bin['Disease_Status'] = df['target'].map({0: 'No_Disease', 1: 'Has_Disease'})

# Xóa các cột số gốc, chỉ giữ lại cột đã phân loại và cột danh mục gốc
cols_numeric_drop = ['age', 'resting_blood_pressure', 'cholestoral', 'Max_heart_rate', 'oldpeak', 'target']
df_categorical = df_bin.drop(columns=cols_numeric_drop)

print("Dữ liệu sau khi rời rạc hóa:")
df_categorical.head()

In [ ]:
# Chuyển đổi sang dạng One-Hot (True/False)
df_encoded = pd.get_dummies(df_categorical)

# Chuyển True/False thành 1/0
df_encoded = df_encoded.astype(int)

print(f"Kích thước ma trận One-Hot: {df_encoded.shape}")
df_encoded.head(3)

In [ ]:
# 1. Tìm tập phổ biến (Frequent Itemsets)
# min_support = 0.2 nghĩa là tổ hợp đó phải xuất hiện ít nhất trong 20% dữ liệu
frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)

# 2. Sinh luật kết hợp
# Lấy các luật có độ Lift >= 1
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# Sắp xếp theo Confidence giảm dần
rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])

print(f"Tìm thấy {len(frequent_itemsets)} tập phổ biến.")
print(f"Sinh được {len(rules)} luật kết hợp.")

# Hiển thị Top 10 luật quan trọng nhất
print("\n--- TOP 10 LUẬT KẾT HỢP (SORT BY CONFIDENCE) ---")
display(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10))

# Lọc các luật dẫn đến bệnh tim (Disease_Status_Has_Disease)
print("\n--- CÁC LUẬT DẪN ĐẾN BỆNH TIM (HAS DISEASE) ---")
disease_rules = rules[rules['consequents'].apply(lambda x: 'Disease_Status_Has_Disease' in x)]
display(disease_rules[['antecedents', 'consequents', 'confidence', 'lift']].head(5))

In [ ]:
X = df_encoded.values

sse = []
silhouettes = []
K_range = range(2, 11)  # Thử số cụm từ 2 đến 10

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)
    sse.append(kmeans.inertia_)
    silhouettes.append(silhouette_score(X, labels))

# Vẽ biểu đồ 2 trục
fig, ax1 = plt.subplots(figsize=(12, 5))

# Biểu đồ Elbow (SSE)
color = 'tab:red'
ax1.set_xlabel('Số lượng cụm (k)')
ax1.set_ylabel('SSE (Inertia)', color=color)
ax1.plot(K_range, sse, marker='o', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.set_title('Đánh giá số cụm tối ưu: Elbow Method & Silhouette Score')

# Biểu đồ Silhouette
ax2 = ax1.twinx()
color = 'tab:blue'
ax2.set_ylabel('Silhouette Score', color=color)
ax2.plot(K_range, silhouettes, marker='s', linestyle='--', color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.grid(True, alpha=0.3)
plt.show()

# Gợi ý k tốt nhất (theo Silhouette cao nhất)
best_k = K_range[np.argmax(silhouettes)]
print(f"Số cụm gợi ý dựa trên Silhouette Score cao nhất: k = {best_k}")

In [ ]:
# Chạy lại KMeans với k tối ưu (Bạn có thể đổi số k theo biểu đồ trên nếu muốn, ví dụ k=2 hoặc k=3)
final_k = best_k 
kmeans_final = KMeans(n_clusters=final_k, random_state=42, n_init=10)
clusters = kmeans_final.fit_predict(X)

# Gán nhãn cụm vào dữ liệu gốc để phân tích
df['Cluster'] = clusters

# Trực quan hóa bằng PCA (Giảm chiều dữ liệu về 2D)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10, 7))
# Vẽ scatter plot
sns.scatterplot(
    x=X_pca[:, 0], 
    y=X_pca[:, 1], 
    hue=df['Cluster'], 
    palette='viridis', 
    style=df['target'], # Hình dáng điểm thể hiện việc có bệnh hay không
    s=100,
    alpha=0.8
)
plt.title(f'Phân cụm K-Means (k={final_k}) trên không gian PCA 2D\n(Màu sắc = Cụm, Hình dáng = Bệnh lý)')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
print("--- THỐNG KÊ ĐẶC ĐIỂM TỪNG CỤM ---")

# 1. Tỉ lệ mắc bệnh trong từng cụm
print("\nTỉ lệ mắc bệnh tim (Target=1) theo từng cụm:")
print(df.groupby('Cluster')['target'].mean().sort_values(ascending=False))

# 2. Đặc điểm trung bình các chỉ số số thực
print("\nGiá trị trung bình các chỉ số theo cụm:")
cols_to_analyze = ['age', 'resting_blood_pressure', 'cholestoral', 'Max_heart_rate']
print(df.groupby('Cluster')[cols_to_analyze].mean())

# 3. Phân bố giới tính trong cụm
print("\nPhân bố giới tính theo cụm:")
print(pd.crosstab(df['Cluster'], df['sex']))